In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir('drive/Colab Notebooks')

### Loading the processed data and training the model

In [22]:
import pandas as pd
data = pd.read_csv('ravi_mod_train.csv') #text in column 1, classifier in column 2.
data=data.dropna(axis=0)
import numpy as np

numpy_array = data.as_matrix()
X = numpy_array[:,0]
Y = numpy_array[:,1]
Y=Y.astype('int')

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


'''
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,2))),('tfidf', TfidfTransformer(use_idf=True)),('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-2, n_iter=5, random_state=19))])
_ = text_clf_svm.fit(X_train, Y_train)
predicted_svm = text_clf_svm.predict(X_test)
print(np.mean(predicted_svm == Y_test))
'''

text_clf = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,2))),('tfidf', TfidfTransformer(use_idf=False)),('clf', MultinomialNB(alpha=0.01))])

text_clf1 = text_clf.fit(X_train,Y_train)

predicted = text_clf1.predict(X_test)
print(np.mean(predicted == Y_test))

'''
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf1, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, Y_train)
print(gs_clf.best_score_)
print(gs_clf.best_params_)
'''
'''
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)
'''

0.8611895892191914


"\nfrom sklearn.model_selection import GridSearchCV\nparameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}\ngs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)\ngs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)\nprint(gs_clf_svm.best_score_)\nprint(gs_clf_svm.best_params_)\n"

### Saving the model

In [0]:

import pickle
#pickle.dump(text_clf1,open('naivebayes85.save','wb'))

### Loading the saved model

In [0]:
loaded=pickle.load(open('naivebayes85.save', 'rb'))
#predicted = loaded.predict(X_test)
#print(np.mean(predicted== Y_test))



In [0]:
import pandas as pd
import time
tick = time.time()
df = pd.read_csv('test_data.csv')
#print(df.head())


### Making the final predictions

In [12]:
numpy_array1 = df.as_matrix()
test_reviews = numpy_array1[:,1]
test_id = numpy_array1[:,0]
#print(test_reviews[0])
final_predict = loaded.predict(test_reviews)
print(final_predict[:100])
print('total time for prediction is %f seconds' % (tick - time.time()))



[2 2 2 0 2 2 1 2 2 2 2 2 0 2 0 2 2 2 1 2 0 2 2 2 2 0 2 2 2 2 0 2 1 2 2 2 2
 2 2 2 1 2 2 2 2 1 2 2 1 2 2 0 2 0 2 2 2 2 2 2 2 1 1 0 2 2 2 2 2 2 1 2 2 0
 0 2 0 2 2 0 2 2 2 2 2 0 0 2 2 2 2 2 2 2 2 0 2 2 2 2]
total time for prediction is -24.264355 seconds


### Creating a .csv file in the required format

In [30]:
submission = pd.DataFrame({'ID':test_id,'label':final_predict})
print(submission.head())


  ID  label
0  1      2
1  2      2
2  3      2
3  4      0
4  5      2


In [0]:
submission.to_csv('group_6_sub69.csv',index=False)